<b>Data Description:</b>
    The file Bank.xls contains data on 5000 customers. The data include customer demographic information (age, income, etc.), the customer's relationship with the bank (mortgage, securities account, etc.), and the customer response to the last personal loan
campaign (Personal Loan). Among these 5000 customers, only 480 (= 9.6%) accepted
the personal loan that was offered to them in the earlier campaign.

<b>Domain:</b>
    Banking

<b>Context:</b>
    This case is about a bank (Thera Bank) whose management wants to explore ways of converting its liability customers to personal loan customers (while retaining them as
depositors). A campaign that the bank ran last year for liability customers showed a
healthy conversion rate of over 9% success. This has encouraged the retail marketing
department to devise campaigns with better target marketing to increase the success
ratio with minimal budget.

<b>Learning Outcomes:</b>

   -  Exploratory Data Analysis
   -  Data Cleaning
   -  Data Visualization
   -  Preparing the data to train a model 
   -  Training and making predictions using a classification model
   -  Model evaluation
    
<b>Objective:</b>
    The classification goal is to predict the likelihood of a liability customer buying personal loans which means we have to build a model which will be used to predict which customer will most likely to accept the offer for personal loan, based on the specific relationship with the bank across various features given in the dataset. Here I will be using the Supervised Learning methods to predict which model is best for this problem amongst Logistic Regresssion, K-Nearest Neighbors(KNN) and Naive Bayes Algorigthm.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### Import the necessary libraries :

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import statsmodels.api as sm

from scipy import stats

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score # Performance measure – Accuracy

from sklearn import preprocessing

<b>Comment :</b> Here I have used numpy, pandas, matplotlib, seaborn, scipy for EDA and Data Visualization. Also used sklearn for data spliting, model building and for confusion matrix. 

## ::--------------------------- Exploratory Data Analysis ---------------------------- ::

#### Read the data as a data frame :- 

In [ ]:
df  = pd.read_csv('/kaggle/input/bank-personal-loan-modellingthera-bank/Bank_Personal_Loan_Modelling.csv')
df.head()

<b>Comment:</b> Here I have read the Personal Loan  dataset using read_csv() function of pandas. df is a dataframe. I have used head() funtion to display first 5 records of the dataset.

<b>Target Column rearrange:- </b> As our Target Column(Personal Loan) is in middle of dataframe so for more convinient I have drop the personal loan column from the original place and appended at last of dataframe.

In [ ]:
personal_loan = df['Personal Loan']
df.drop(['Personal Loan'], axis=1, inplace = True)
df['Personal Loan'] = personal_loan
df.head(5)

<b>Comment:</b> Now the traget column is appended at the end of the dataframe.

<b> Features(attributes) Understanding from the above dataframe :- </b> 

- The ID variable can be ignored as it will not any effect on our model. As we know customer Id is just to maitain the record in serial order. There is no relationship with Id and Loan.
- Target Variable is <b>Personal Loan</b> which describe whether the person has taken loan or not. This is the variable which we need to predict.

Nonimal Varibles :
- ID - Customer ID
- ZIP Code - Home Address ZIP code of the customer. This variable can also be ignored becasue we can not judge the customers based on thier area or location.

Ordinal Categorical variables :
- Family - Number of famlily member of the customer
- Education - Education level of the customer. In our dataset it ranges from 1 to 3 which are Under Graduate, Graduate and Post Graduate respectivly.

Interval Variables :
- Age        - Age of the customer
- Experience - Years of experience of customer has
- Income     - Annula Income of the customer which is in dollars
- CCAvg      - Avg. spending on credit cards per month which in dollars.
- Mortgage    - Value of House Mortgage

Binary Categorical Variable :
- CD Account - Does the customer have CD Account with bank or not?
- Security Account - Does the customer have Security Account with bank or not?
- Online  - Does the customer have Online banking facility with bank or not?
- Credit Card - Does the customer have a credit card issued by Universal Bank or not?
- Personal Loan - This our target variable which we have to predict. This indicates that the customer has token loan or not?





















### Shape of the data :- 

In [ ]:
rows_count, columns_count = df.shape
print('Total Number of rows :', rows_count)
print('Total Number of columns :', columns_count)

<b>Comment:</b> Shape of the dataframe is (5000, 14).
There are 5000 rows and 14 columns in the dataset. 

###  Data type of each attribute :-

In [ ]:
df.dtypes

<b>comment : </b> We can also display the data types of dataframe using df.info() function which gives even more useful info.

In [ ]:
df.info()

<b>Comment :</b> Here we can see that all the variables are numerical. But the columns 'CD Account', 'Online', 'Family', 'Education' , 'CreditCard' and 'Securities Account' are categorical variable which should be in 'category' type. 

### Checking the presence of missing values :-

In [ ]:
df.isnull().sum() 

<b>Comment:</b> Here total missing values count from each column is 0 and we can see there is no missing value in the dataframe.

In [ ]:
df.isnull().values.any()

<b>Comment:</b> This gives if we have any missing values at all. False : No missing value, True: Missing value 

#### Missing value Visualization :- 

In [ ]:
sns.heatmap(df.isna(), yticklabels=False, cbar=False, cmap='viridis')

<b>Comment: </b> From the above heatmap graph we can see that there is no missing value in dataset.

### Checking the unique data :-

In [ ]:
df.nunique()

### Transposing index and columns:-

In [ ]:
df.describe().T

<b>Observation:</b> 
- <b>Important : </b>Here we can see that minumum value of Experience column is <b>-3.0</b> which could be a mistake because Experience can not be negative. So I will be fixing it in data cleaning and error fixing part.
- Binary varibales 'Personal Loan', 'Credit Card', 'Online', 'CD Account', 'Security Account' has clean data.
- Ordinary Cat variables 'Familty' and 'Education' are also clean.
- Target variable also looks fine. 

In [ ]:
### Five point summary of  attributes and label :-

In [ ]:
df_transpose = df.describe().T
df_transpose[['min', '25%', '50%', '75%', 'max']]

###  Pair plot that includes all the columns of the data frame :-

<b>Note: </b> I am not using ID column in tha pair plot as it is not relevent with our analysis. Id column is jut for record index.

In [ ]:
sns.pairplot(df.iloc[:,1:]) 

<b>Observation : </b> From the above pair plot we can infer the association among the attributes and target column as follows:
- 'Age' column is normally distributed. Here can see that the mean and midean is almost same which we have also seen in the transpose matrics. Most of the customers age is between 25 to 65 years.
- 'Experience' is also mormally distributed. Here also mean is amost equal to midean. 'Experience' and 'Age' are stong possitive associasion.
- 'Income' is positively skewed and it will also have the outlier
- We dont see any relationship with the ZIP Code and other variables.
- Family and Education has low association with the 'Personal Loan'. 
- The disribution of CCAvg is also a possotively skewed variable. Majority of the customers average monthly spending is between 1k to 9k.
- 'Mortgage' is also positively skewed. Majority of the individuals have a mortgage of less than 40K. 

 ### Error Fixing(Data Cleaning) :-

In [ ]:
# Checking the negative values
df[df['Experience'] < 0]['Experience'].value_counts()

In [ ]:
# Total records of negative experience
df[df['Experience'] < 0]['Experience'].count()

<b>Comment:</b> There are 52 records with negative Experience are present in the dataset.



#### Checking the association of Experience with other quantitive variables :

In [ ]:
quantitiveVar = ['Age', 'Income', 'Income', 'CCAvg', 'Mortgage']
expGrid = sns.PairGrid(df, y_vars = 'Experience', x_vars = quantitiveVar)
expGrid.map(sns.regplot)

<b>Observation:</b> By looking above plots we can see that 'Age' has very strong and possitive association with 'Experience'. I am also considering 'Education' to fix the negative experience error. Becuase as we know experience relats to the education level. 

<b>Decission: </b> We can replace each negative 'Experience' value with the median of possitive 'Experience' associated with the particular 'Age' and 'Education' value.

<b>Steps to be followed in the following code: </b> 
- Get the record of experience whose value is greater than 0
- Get the record of experience whose value is lesser than 0
- Get the list of Cutomer ID whose experience value is negative
- Get the list of 'Age' values where it finds negative values in 'Experience' column
- Get the list of 'Education' values where it finds negaitve values in 'Experience' column
- Next it filters the records matching the above conditions from the dataframe which has data with possive experience and takes the median and store in exp. There could be chance that there will be no possive experience which matches the above condtion. In such case it matches the above condiiton from the dataframe which has the record with negative experience and gets the median and store in exp.
- Next it will replace the negative experience with the median.
- After execution of below code we will check the negative experience again.

In [ ]:
df_Possitive_Experience = df[df['Experience'] > 0]
df_Negative_Experience =  df[df['Experience'] < 0]
df_Negative_Experience_List = df_Negative_Experience['ID'].tolist()

for id in df_Negative_Experience_List:
    age_values = df.loc[np.where(df['ID']==id)]["Age"].tolist()[0]
    education_values = df.loc[np.where(df['ID']==id)]["Education"].tolist()[0]
    possitive_Experience_Filtered = df_Possitive_Experience[(df_Possitive_Experience['Age'] == age_values) & (df_Possitive_Experience['Education'] == education_values)]
    if possitive_Experience_Filtered.empty :
        negative_Experience_Filtered = df_Negative_Experience[(df_Negative_Experience['Age'] == age_values) & (df_Negative_Experience['Education'] == education_values)]
        exp = round(negative_Experience_Filtered['Experience'].median())
    else:
        exp = round(possitive_Experience_Filtered['Experience'].median())
    df.loc[df.loc[np.where(df['ID']==id)].index, 'Experience'] = abs(exp)

In [ ]:
# Total records of negative experience
df[df['Experience'] < 0]['Experience'].count()

<b> Comment: </b> Now we can see that negative values count is 0 means there is no negative value anymore in the dataframe.
- Describing the 'Experience' column to check the count, mean, standard deviation and five point summary.

In [ ]:
df.Experience.describe()

<b>Comment: </b> Now we can see the min is 0.0 which was -3.0 before error fixing.   

## ::----------------------------------- Data Visualization --------------------------------::

###  Data distribution in each attribute :-

In [ ]:
sns.distplot(df['ID'])

<b>Observation:</b> The above graph is unform distribution. 

In [ ]:
sns.distplot(df['Age'])

<b>Observation:</b> Age column is normaly distrubuted.

In [ ]:
sns.distplot(df['Experience'])

<b>Observation:</b>  Experience is also normaly distributed.

In [ ]:
sns.distplot(df['Income'])

<b> Observation: </b>The above distributionis is right skewed distribution because the tail goes to the right.

In [ ]:
sns.distplot(df['ZIP Code'])

<b>Observation:</b> The is uniformaly distrubuted. Data points are more with family size 1 and 2.

In [ ]:
sns.distplot(df['CCAvg'])

<b> Comment: </b> The above distributionis is right skewed distribution because the tail goes to the right. Most of the customers monthly avg. spending on credit cards is between 1k to 2.5K. There are very few customers whose monthly avg. spending on credit card is more than 8k. 
    

In [ ]:
sns.distplot(df['Education'])

<b>Observation:</b> Undergrad level customers are more than the Graduate and Advanced/Professional customers.
 

In [ ]:
sns.distplot(df['Mortgage'])

<b>Observation:</b>The above distributionis is right skewed distribution because the tail goes to the right. 
Most of the customers do not have mortgage. There are more customers whose mortgage amount is between $80000 to $150000 . Very few customers whos mortgage amount is more than $600000.

In [ ]:
sns.distplot(df['Online'])

<b>Observation:</b> This is a Bernoulli Distrubution. Number of customers who have Online accout is geater than the number of customers who do not have online account

In [ ]:
sns.distplot(df['CreditCard'])

<b>Observation:</b> It is a Bernoulli Distrubution. Number of customers without Credit Card is almost double than the number of customers with Credit Card.

###  Target column distribution and Data Visualization with Personal Loan Column :-

In [ ]:
loan_counts = pd.DataFrame(df["Personal Loan"].value_counts()).reset_index()
loan_counts.columns =["Labels","Personal Loan"]
loan_counts

<b>Comment:</b> Out of 5000 data points, 4520 are labeled as 0 and 480 as 1. Percentage of customers who took loan is significantlly greater than customers who did not take loan. I have also show the percentage using in pie chart below.


In [ ]:
fig1, ax1 = plt.subplots()
explode = (0, 0.15)
ax1.pie(loan_counts["Personal Loan"], explode=explode, labels=loan_counts["Labels"], autopct='%1.1f%%',
        shadow=True, startangle=70)
ax1.axis('equal')  
plt.title("Personal Loan Percentage")
plt.show()

<b>Important :</b> From the above we can see that the data is having a huge bias(alomot 1:10) towards the category of people not accepting the personal loan. Hence we can build an opinion that our model will tends to perform better towards predicting which customers will not accept the personal loan. However, our goal is to identityfy the customer who can accept the personal loan based on the given features. 

#### Influence of important features on Personal Loan. From the above pair plot we can see some features which has relationship with the target column i.e personal loan :-

In [ ]:
sns.catplot(x='Family', y='Income', hue='Personal Loan', data = df, kind='swarm')

<b>Observation:</b> Customers who have family size 3 or greater with higher income between 100k to 200k are more likely to take loan.

In [ ]:
sns.boxplot(x='Education', y='Income', hue='Personal Loan', data = df)

<b>observation: </b> From above we can say that customers with undergraduate level of education and family greater than 3 are good customers who took loan. Customer who took loan have same income range irrespective of education level. Education of Graduate and above have more chance to take loan.

In [ ]:
sns.boxplot(x="Education", y='Mortgage', hue="Personal Loan", data=df)

<b>Observation :</b> From the above box plot we can see that customers whose education level is 1 and did not take loan has higher mortgage than customers who take loan of same education level. Customers whose education level is 2and 3 and did not take loan has lesser mortgage than customers who take loan of same education level. 

In [ ]:
sns.countplot(x="Securities Account", data=df,hue="Personal Loan")

<b>Observation :</b> Customers who has securies account are more likly to take loan. Majority of customers who does not have loan do not have securities account. 

In [ ]:
sns.countplot(x='Family',data=df,hue='Personal Loan')

<b>Observation :</b> Family size does not have any impact in personal loan. But it seems families with size of 3 and 4 are more likely to take loan.

In [ ]:
sns.countplot(x='CD Account',data=df,hue='Personal Loan')

<b>Observation: </b> Customers who does not have CD account , does not have loan as well. This seems to be majority. But almost all customers who has CD account has loan as well

In [ ]:
sns.boxplot(x="CreditCard", y='CCAvg', hue="Personal Loan", data=df)


<b>Observation: </b> Customers who have credit card and monthly spending is higher are more likly to take loan.

In [ ]:
sns.catplot(x='Age', y='Experience', hue='Personal Loan', data = df, height=8.27, aspect=11/5)

<b>Observation: </b>From the above we clearly see that  Age and Experience have very storng association. As Age increases Experience also increases. We can try building our model by droping Experience column. 

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(df[df["Personal Loan"] == 0]['CCAvg'], color = 'r',label='Personal Loan=0')
sns.distplot(df[df["Personal Loan"] == 1]['CCAvg'], color = 'b',label='Personal Loan=1')
plt.legend()
plt.title("CCAvg Distribution")

<b>Observation: </b> Customers who have taken personal loan have higher credit card average than those who did not take loan. So high credit card average seems to be good predictor of whether or not a customer will take a personal loan.

In [ ]:
print('Credit card spending of Non-Loan customers: ',df[df['Personal Loan'] == 0]['CCAvg'].median()*1000)
print('Credit card spending of Loan customers    : ', df[df['Personal Loan'] == 1]['CCAvg'].median()*1000)

<b>Comment:</b> The graph show persons who have personal loan have a higher credit card average. Average credit card spending with a median of 3800 dollar indicates a higher probability of personal loan. Lower credit card spending with a median of 1400 dollars is less likely to take a loan.

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(df[df["Personal Loan"] == 0]['Income'], color = 'r',label='Personal Loan=0')
sns.distplot(df[df["Personal Loan"] == 1]['Income'], color = 'b',label='Personal Loan=1')
plt.legend()
plt.title("Income Distribution")

<b>Observation: </b> Customers who have taken personal loan have income than those who did not take. So high income seems to be good predictor of whether or not a customer will take a personal loan.

### Outliers Detection :-

In [ ]:
df.boxplot(return_type='axes', figsize=(20,5))

<b>Comment:</b> From the above boxplot we can see there are outliers on few colomns. Mortgage has more number of outlier. Income, CCAvg have also outliers. We will try to fix the outlier by scaling the attributes.

### Correlation using Heatmap :-

In [ ]:
plt.figure(figsize = (15,7))
plt.title('Correlation of Attributes', y=1.05, size=19)
sns.heatmap(df.corr(), cmap='plasma',annot=True, fmt='.2f')

<b>Observation:</b> From the above heatmap we can see that :
- Age and Experience are highly correlated and the correlation is almost 1.
- 'Income' and 'CCAvg' is moderately correlated.
- Personal Loan has maximum correlation with 'Income', 'CCAvg', 'CD Account', 'Mortgage', and 'Education'.
- We can see in above heat map there is association of 'CD Account' with 'Credit Card', 'Securities Account', 'Online', 'CCAvg' and 'Income'.
- 'Mortgage' has moderate correlation with 'Income' which is about 12%.
- 'Income' influences 'CCAvg', 'Personal Loan', 'CD Account' and 'Mortgage'.


# ::------------------------- Model Building ---------------------------------::

## Dimensionality Reduction : -
- As we have seen above 'ID' and 'ZIP Code' are not relevent for our model building so we will drop it.
- 'Age' and 'Experience' are highly correlated so we will build our model <b>with 'Experience'</b> and <b>without 'Experience'</b> after that we will compare the accurace which will lead us to the conclution that with 'Experience' or without 'Experience' which model is better for prediction. 

#### Dropping 'ID' and 'ZIP Code' :

In [ ]:
df.head(1)

In [ ]:
df = df.drop(['ID','ZIP Code'], axis=1)

In [ ]:
df.head(1)

### Creating two dataframes with 'Experience' and without 'Experience' repectively :

In [ ]:
loan_with_experience = df
loan_without_experience = df.drop(['Experience'], axis=1)

In [ ]:
print('Columns With Experience : ', loan_with_experience.columns)
print('Columns Without Experience : ', loan_without_experience.columns)

#### Seperating Target Variable from Independent Variables from Expr and Wihtout Expr dataframe :


In [ ]:
# From Exprenece Dataframe:
X_Expr = loan_with_experience.drop('Personal Loan', axis=1)
Y_Expr = loan_with_experience[['Personal Loan']]

In [ ]:
# From Exprenece Dataframe:
X_Without_Expr = loan_without_experience.drop('Personal Loan', axis=1)
Y_Without_Expr = loan_without_experience[['Personal Loan']]

## Spliting the data into training and test set in the ratio of 70:30 respectively :-

In [ ]:
# From Experience Dataframe:
X_Expr_train, X_Expr_test, y_Expr_train, y_Expr_test = train_test_split(X_Expr, Y_Expr, test_size=0.30, random_state=1)
print('x train data {}'.format(X_Expr_train.shape))
print('y train data {}'.format(y_Expr_train.shape))
print('x test data  {}'.format(X_Expr_test.shape))
print('y test data  {}'.format(y_Expr_test.shape))

In [ ]:
# From Without Experience Dataframe:
X_train, X_test, y_train, y_test = train_test_split(X_Without_Expr, Y_Without_Expr, test_size=0.30, random_state=1)
print('x train data {}'.format(X_train.shape))
print('y train data {}'.format(y_train.shape))
print('x test data  {}'.format(X_test.shape))
print('y test data  {}'.format(y_test.shape))

## ::---------------------------- Logistic Regression ---------------------------------------::

### With Experience Column:-

In [ ]:
#X_Exp_train, X_Exp_test, y_Exp_train, y_Exp_test
logreg_expr_model = LogisticRegression()
logreg_expr_model.fit(X_Expr_train, y_Expr_train)
print(logreg_expr_model , '\n')

# Predicting for test set
logreg_expr_y_predicted = logreg_expr_model.predict(X_Expr_test)
logreg_expr_score = logreg_expr_model.score(X_Expr_test, y_Expr_test)
logreg_expr_accuracy = accuracy_score(y_Expr_test, logreg_expr_y_predicted)

logestic_confusion_matrix_expr = metrics.confusion_matrix(y_Expr_test, logreg_expr_y_predicted)

### Without Experience Column:-

In [ ]:
#X_train, X_test, y_train, y_test
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

# Predicting for test set
logreg_y_predicted = logreg_model.predict(X_test)
logreg_score = logreg_model.score(X_test, y_test)
logreg_accuracy = accuracy_score(y_test, logreg_y_predicted)
logestic_confusion_matrix = metrics.confusion_matrix(y_test, logreg_y_predicted)

<b>Comparison :</b> Below is the comparison b/w Logistic Regression Model Accuracy and Confussion Matrix with 'Experience' and W/O 'Experience'.

In [ ]:
# Accuracy
print('Logistic Regression Model Accuracy Score W/O Experience  : %f'  % logreg_accuracy)
print('Logistic Regression Model Accuracy Score With Experience : %f'  % logreg_expr_accuracy)

# Confusion Matrix
print('\nLogistic Regression Confusion Matrix W/O Experience: \n', logestic_confusion_matrix)
print('\nTrue Possitive    = ', logestic_confusion_matrix[1][1])
print('True Negative     = ',   logestic_confusion_matrix[0][0])
print('False Possive     = ',   logestic_confusion_matrix[0][1])
print('False Negative    = ',   logestic_confusion_matrix[1][0])
print('\nLogistic Regression Confusion Matrix With Experience: \n', logestic_confusion_matrix_expr)
print('\nTrue Possitive    = ', logestic_confusion_matrix_expr[1][1])
print('True Negative     = ',   logestic_confusion_matrix_expr[0][0])
print('False Possive     = ',   logestic_confusion_matrix_expr[0][1])
print('False Negative    = ',   logestic_confusion_matrix_expr[1][0])


<b>Observation: </b> 
- From the above accuracy results we see that accuracy is higher with 'Experience' (94.60 %) than without 'Experience' (94.26%). 
- Also from the above confusion metrices we can see that the prediction of customers who dont accept loan and the customers who accept loan is better with 'Experience'.
- <b>Type 1 (False Possitive)</b> and <b>Type 2(False Negative)</b> errors is less with experience.
- Hence we can imporove the accuracy by scalling the attributes.
- We will not consider dafaframe without 'Experience' for further iteration.

### Improvement of  the model -------- Iteration 2 For Logistic Regression with Experience--------

In [ ]:
#X_Expr_train, X_Expr_test, y_Expr_train, y_Expr_test
X_train_scaled = preprocessing.scale(X_Expr_train)
X_test_scaled = preprocessing.scale(X_Expr_test)

In [ ]:
scaled_logreg_model = LogisticRegression()
scaled_logreg_model.fit(X_train_scaled, y_Expr_train)

# Predicting for test set
scaled_logreg_y_predicted = scaled_logreg_model.predict(X_test_scaled)
scaled_logreg_model_score = scaled_logreg_model.score(X_test_scaled, y_Expr_test)
scaled_logreg_accuracy = accuracy_score(y_Expr_test, scaled_logreg_y_predicted)

scaled_logreg_confusion_matrix = metrics.confusion_matrix(y_Expr_test, scaled_logreg_y_predicted)


In [ ]:

print('----------------------Final Analysis of Logistic Regression----------------------------\n')
print('After Scalling Logistic Regression Model Accuracy Score with Experience: %f'  % scaled_logreg_accuracy)
print('\nAfter Scalling Logistic Regression Confusion Matrix With Experience: \n', scaled_logreg_confusion_matrix)
print('\nTrue Possitive    = ', scaled_logreg_confusion_matrix[1][1])
print('True Negative     = ',   scaled_logreg_confusion_matrix[0][0])
print('False Possive     = ',   scaled_logreg_confusion_matrix[0][1])
print('False Negative    = ',   scaled_logreg_confusion_matrix[1][0])
print('\nK-NN classification Report : \n',metrics.classification_report(y_Expr_test, scaled_logreg_y_predicted))
conf_table = scaled_logreg_confusion_matrix
a = (conf_table[0,0] + conf_table[1,1]) / (conf_table[0,0] + conf_table[0,1] + conf_table[1,0] + conf_table[1,1])
p = conf_table[1,1] / (conf_table[1,1] + conf_table[0,1])
r = conf_table[1,1] / (conf_table[1,1] + conf_table[1,0])
f = (2 * p * r) / (p + r)
print("Accuracy of accepting Loan  : ",round(a,2))
print("precision of accepting Loan : ",round(p,2))
print("recall of accepting Loan    : ",round(r,2))
print("F1 score of accepting Loan  : ",round(f,2))

## ::------------------------------ K-NN --------------------------------------------::

### Steps to be followed:-
- We will follow the same procedures as we have followed in Logistic Regression.
- We will try the model with 'Experience' and without 'Experience'.
- will run the KNN with number of odd neighbours ranges from 1 to 20 and will find the optimal number of neighbours using the Mis classification error.
- After finding <b>best K</b>, will build the model using 'Experience' and without 'Experience'. 

#### Below Code is to find the best K(Neighbors)


In [ ]:
#Creating number list from range 1 to 20 of K for KNN

numberList = list(range(1,20))
neighbors = list(filter(lambda x: x % 2 != 0 , numberList)) #subsetting just the odd ones

#Declearing a empty list that will hold the accuracy scores
ac_scores = []
#performing accuracy metrics for value from 1,3,5....19
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    #predict the response
    knn.fit(X_train, y_train.values.ravel())               
    y_pred = knn.predict(X_test)
    #evaluate accuracy
    scores = accuracy_score(y_test, y_pred)
    #insert scores to the list
    ac_scores.append(scores)                

MSE = [1 - x for x in ac_scores] # changing to misclassification error


# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]

print('Odd Neighbors : \n', neighbors)
print('\nAccuracy Score : \n', ac_scores)
print('\nMisclassification error :\n', MSE)
print("\nThe optimal number of neighbor is k=",optimal_k)

# plot misclassification error vs k
plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

<b>Decission: </b> 
- From above result we can see that the misclassification error is minimum at <b>k = 3</b>. 
- Hence I am considering 3 is the optimal k. When k=3 the model accuracy is 0.909.
- From above graph of misclassification error vs k (with k value on X-axis) we can also see that error is very low when K is 3.

### Model building using  'Without Experience' dataframe:-
- We have already created the dataframe and splited the data logistic regression, will be using the same.
- Splited data from 'Without Experience' dataframe: X_train, X_test, y_train, y_test

In [ ]:
# instantiating learning model (optimal_k = 3)
knn_model = KNeighborsClassifier(n_neighbors=optimal_k , weights = 'uniform', metric='euclidean')
knn_model.fit(X_train, y_train)
knn_y_predicted = knn_model.predict(X_test)
knn_score = knn_model.score(X_test, y_test)
knn_accuracy = accuracy_score(y_test, knn_y_predicted)
knn_confusion_matrix = metrics.confusion_matrix(y_test, knn_y_predicted)

### Model building using  'With Experience' dataframe:-
- We have already created the dataframe and splited the data logistic regression, will be using the same.
- Splited data from 'With Experience' dataframe: X_Expr_train, X_Expr_test, y_Expr_train, y_Expr_test

In [ ]:
# instantiating learning model (optimal_k = 3)
knn_model_expr = KNeighborsClassifier(n_neighbors=optimal_k , weights = 'uniform', metric='euclidean')
knn_model_expr.fit(X_Expr_train, y_Expr_train)
knn_expr_y_predicted = knn_model_expr.predict(X_Expr_test)
knn_expr_score = knn_model_expr.score(X_Expr_test, y_Expr_test)
knn_expr_accuracy = accuracy_score(y_Expr_test, knn_expr_y_predicted)
knn_confusion_matrix_expr = metrics.confusion_matrix(y_Expr_test, knn_expr_y_predicted)

<b>Comparison :</b> Below is the comparison b/w K-NN Model Accuracy and Confussion Matrix with 'Experience' and W/O 'Experience'.

In [ ]:
# Comparison 
print('K-NN Model Accuracy Score W/O Experience  : %f'  % knn_accuracy)
print('K-NN Model Accuracy Score With Experience : %f'  % knn_expr_accuracy)

# Confusion Matrix
print('\nK-NN Confusion Matrix W/O Experience: \n', knn_confusion_matrix)
print('\nTrue Possitive    = ', knn_confusion_matrix[1][1])
print('True Negative     = ',   knn_confusion_matrix[0][0])
print('False Possive     = ',   knn_confusion_matrix[0][1])
print('False Negative    = ',   knn_confusion_matrix[1][0])
print('\nK-NN Confusion Matrix With Experience: \n', knn_confusion_matrix_expr)
print('\nTrue Possitive    = ', knn_confusion_matrix_expr[1][1])
print('True Negative     = ',   knn_confusion_matrix_expr[0][0])
print('False Possive     = ',   knn_confusion_matrix_expr[0][1])
print('False Negative    = ',   knn_confusion_matrix_expr[1][0])


<b>Observation: </b> 
- From the above accuracy results we see that accuracy is higher at without 'Experience' (90.93%) than with 'Experience' (90.20%). 
- Also from the above confusion metrices we can see that the prediction of customers who dont accept loan and the customers who accept loan is better at without 'Experience'.
- <b>Type 1 (False Possitive)</b> and <b>Type 2(False Negative)</b> errors is less at without Experience.
- Hence we can imporove the accuracy by scalling the attributes.
- We will not consider dafaframe 'With Experience' for further iteration.

### Improvement of  the model -------- Iteration 2 For K-NN without Experience dataset--------

In [ ]:
#X_train, X_test, y_train, y_test
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

In [ ]:
scaled_knn_model = KNeighborsClassifier(n_neighbors=optimal_k , weights = 'uniform', metric='euclidean')
scaled_knn_model.fit(X_train_scaled, y_train)
scaled_knn_y_predict = scaled_knn_model.predict(X_test_scaled)
scaled_knn_score = scaled_knn_model.score(X_test_scaled, y_test)
scaled_knn_accuracy = accuracy_score(y_test, scaled_knn_y_predict)
scaled_knn_confusion_matrix = metrics.confusion_matrix(y_test, scaled_knn_y_predict)

In [ ]:

print('----------------------Final Analysis of K-NN----------------------------\n')
print('After Scalling K-NN Model Accuracy Score without Experience: %f'  % scaled_knn_accuracy)
print('\nAfter Scalling K-NN Confusion Matrix Without Experience: \n', scaled_knn_confusion_matrix)
print('\nTrue Possitive    = ', scaled_knn_confusion_matrix[1][1])
print('True Negative     = ',   scaled_knn_confusion_matrix[0][0])
print('False Possive     = ',   scaled_knn_confusion_matrix[0][1])
print('False Negative    = ',   scaled_knn_confusion_matrix[1][0])
print('\nK-NN classification Report : \n',metrics.classification_report(y_test, scaled_knn_y_predict))
knn_conf_table = scaled_knn_confusion_matrix
a = (knn_conf_table[0,0] + knn_conf_table[1,1]) / (knn_conf_table[0,0] + knn_conf_table[0,1] + knn_conf_table[1,0] + knn_conf_table[1,1])
p = knn_conf_table[1,1] / (knn_conf_table[1,1] + knn_conf_table[0,1])
r = knn_conf_table[1,1] / (knn_conf_table[1,1] + knn_conf_table[1,0])
f = (2 * p * r) / (p + r)
print("\nAccuracy of accepting Loan  : ",round(a,2))
print("precision of accepting Loan : ",round(p,2))
print("recall of accepting Loan    : ",round(r,2))
print("F1 score of accepting Loan  : ",round(f,2))

## ::-------------------------- Naïve Bayes -------------------------------------::


### Steps to be followed:-
- We will follow the same procedures as we have done in above models.
- We will try the model with 'Experience' and without 'Experience'.

### Model building using  'Without Experience' dataframe:-
- Splited data from 'Without Experience' dataframe: X_train, X_test, y_train, y_test

In [ ]:
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)
gnb_y_predicted = gnb_model.predict(X_test)
gnb_score = gnb_model.score(X_test, y_test)
gnb_accuracy = accuracy_score(y_test, gnb_y_predicted)
gnb_confusion_matrix = metrics.confusion_matrix(y_test, gnb_y_predicted)

### Model building using  'With Experience' dataframe:-
- Splited data from 'With Experience' dataframe: X_Expr_train, X_Expr_test, y_Expr_train, y_Expr_test

In [ ]:
gnb_expr_model = GaussianNB()
gnb_expr_model.fit(X_Expr_train, y_Expr_train)
gnb_expr_y_predicted = gnb_expr_model.predict(X_Expr_test)
gnb_expr_score = gnb_expr_model.score(X_Expr_test, y_Expr_test)
gnb_expr_accuracy = accuracy_score(y_Expr_test, gnb_expr_y_predicted)
gnb_expr_confusion_matrix = metrics.confusion_matrix(y_Expr_test, gnb_expr_y_predicted)

In [ ]:
# Comparison 
print('Naïve Bayes Model Accuracy Score W/O Experience  : %f'  % gnb_accuracy)
print('Naïve Bayes Model Accuracy Score With Experience : %f'  % gnb_expr_accuracy)

# Confusion Matrix
print('\nNaïve Bayes Confusion Matrix W/O Experience: \n', gnb_confusion_matrix)
print('\nTrue Possitive    = ', gnb_confusion_matrix[1][1])
print('True Negative     = ',   gnb_confusion_matrix[0][0])
print('False Possive     = ',   gnb_confusion_matrix[0][1])
print('False Negative    = ',   gnb_confusion_matrix[1][0])
print('\nNaïve Bayes Confusion Matrix With Experience: \n', gnb_expr_confusion_matrix)
print('\nTrue Possitive    = ', gnb_expr_confusion_matrix[1][1])
print('True Negative     = ',   gnb_expr_confusion_matrix[0][0])
print('False Possive     = ',   gnb_expr_confusion_matrix[0][1])
print('False Negative    = ',   gnb_expr_confusion_matrix[1][0])



<b>Observation: </b> 
- From the above accuracy results we see that accuracy is alomost at without 'Experience' and with 'Experience'. 
- Also from the above confusion metrices we can see that the prediction of customers who dont accept loan and the customers who accept loan is almost same at without 'Experience'.
- <b>Type I (False Possitive)</b> and <b>Type II(False Negative)</b> errors is same.
- Hence we can imporove the accuracy by scalling the attributes.
- We can consider any dafaframe 'With Experience' or 'Without Experience' for further iteration.

### Improvement of  the model -------- Iteration 2 For Naïve Bayes without Experience dataset--------
- We have already scaled the attribute in K-NN model building, will be using the same
- X_train_scaled and X_test_scaled

In [ ]:
scaled_gnb_model = GaussianNB()
scaled_gnb_model.fit(X_train_scaled, y_train)
scaled_gnb_y_predict = scaled_gnb_model.predict(X_test_scaled)
scaled_gnb_score = scaled_gnb_model.score(X_test_scaled, y_test)
scaled_gnb_accuracy = accuracy_score(y_test, scaled_gnb_y_predict)
scaled_gnb_connfusion_matrix = metrics.confusion_matrix(y_test, scaled_gnb_y_predict)

In [ ]:

print('----------------------Final Analysis of Naïve Bayes----------------------------\n')
print('After Scalling Naïve Bayes Model Accuracy Score: %f'  % scaled_gnb_accuracy)
print('\nAfter Scalling Naïve Bayes Confusion Matrix: \n', scaled_gnb_connfusion_matrix)
print('\nTrue Possitive    = ', scaled_gnb_connfusion_matrix[1][1])
print('True Negative     = ',   scaled_gnb_connfusion_matrix[0][0])
print('False Possive     = ',   scaled_gnb_connfusion_matrix[0][1])
print('False Negative    = ',   scaled_gnb_connfusion_matrix[1][0])
print('\n Gaussian Naive Bayes classification Report : \n',metrics.classification_report(y_test, gnb_y_predicted))
gnb_conf_table = scaled_gnb_connfusion_matrix
a = (gnb_conf_table[0,0] + gnb_conf_table[1,1]) / (gnb_conf_table[0,0] + gnb_conf_table[0,1] + gnb_conf_table[1,0] + knn_conf_table[1,1])
p = gnb_conf_table[1,1] / (gnb_conf_table[1,1] + gnb_conf_table[0,1])
r = gnb_conf_table[1,1] / (gnb_conf_table[1,1] + gnb_conf_table[1,0])
f = (2 * p * r) / (p + r)
print("\nAccuracy of accepting Loan   : ",round(a,2))
print("precision of accepting Loan  : ",round(p,2))
print("recall of accepting Loan     : ",round(r,2))
print("F1 score of accepting Loan   : ",round(f,2))

# =========== COMPARISON OF ABOVE THREE MODELS ==========

In [ ]:
print('Overall Model Accuracy After scaling:\n')
print ('Logistic Regression : {0:.0f}%'. format(scaled_logreg_accuracy * 100))
print ('K-Nearest Neighbors : {0:.0f}%'. format(scaled_knn_accuracy * 100))
print ('Naive Bayes         : {0:.0f}%'. format(scaled_gnb_accuracy * 100))

print('\nOverall Model Confusion matrix After scaling:\n')
print('\nLogistic Regression: \n', scaled_logreg_confusion_matrix)
print('\n     True Possitive    = ', scaled_logreg_confusion_matrix[1][1])
print('     True Negative     = ',   scaled_logreg_confusion_matrix[0][0])
print('     False Possive     = ',   scaled_logreg_confusion_matrix[0][1])
print('     False Negative    = ',   scaled_logreg_confusion_matrix[1][0])

print('\nK-Nearest Neighbors: \n', scaled_knn_confusion_matrix)
print('\n    True Possitive    = ', scaled_knn_confusion_matrix[1][1])
print('    True Negative     = ',   scaled_knn_confusion_matrix[0][0])
print('    False Possive     = ',   scaled_knn_confusion_matrix[0][1])
print('    False Negative    = ',   scaled_knn_confusion_matrix[1][0])

print('\nNaive Bayes: \n', scaled_gnb_connfusion_matrix)
print('\n    True Possitive    = ', scaled_gnb_connfusion_matrix[1][1])
print('    True Negative     = ',   scaled_gnb_connfusion_matrix[0][0])
print('    False Possive     = ',   scaled_gnb_connfusion_matrix[0][1])
print('    False Negative    = ',   scaled_gnb_connfusion_matrix[1][0])


print('\n\nReceiver Operating Characteristic (ROC) curve to evalute the classifier output quality.  If area of curve is closer to 1 which means better the model and if area of curve is closer to 0 which means poor the model.')

knn_fpr, knn_tpr, knn_threshold = metrics.roc_curve(y_test, scaled_knn_y_predict)
knn_roc_auc = metrics.roc_auc_score(y_test, scaled_knn_y_predict)
fig1_graph = plt.figure(figsize=(15,4))
fig1_graph.add_subplot(1,3,1)
plt.plot(knn_fpr, knn_tpr, label='KNN Model (area = %0.2f)' % knn_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")


logistic_fpr, logistic_tpr, logistic_threshold = metrics.roc_curve(y_Expr_test, scaled_logreg_y_predicted)
logistic_roc_auc = metrics.roc_auc_score(y_Expr_test, scaled_logreg_y_predicted)
fig1_graph.add_subplot(1,3,2)
plt.plot(logistic_fpr, logistic_tpr, label='Logistic Model (area = %0.2f)' % logistic_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")

nb_fpr, nb_tpr, nb_threshold = metrics.roc_curve(y_test, scaled_gnb_y_predict)
nb_roc_auc = metrics.roc_auc_score(y_test, scaled_gnb_y_predict)
fig1_graph.add_subplot(1,3,3)
plt.plot(nb_fpr, nb_tpr, label='Naive-Bayes Model (area = %0.2f)' % nb_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


### Final Conclusion to Predict the best Model ::-

- From the above, we can clearly see that k-Nearest Neighbors Alogorithm with scaled data gives us best accuracy of 96%. 
- Also the <b>Type I(False Posssitive)</b> and <b>Type II(False Negative)</b> errors are least in K-Nearest model. 
- The area in ROC curve for K-NN is 0.82 which is close to 1 which stats that K-NN is the best model in comparesion of Logistic Model and Naive-Bayes Model whose ROC area is 0.79 and 0.73 repectively.
- Hence among the above three algorithm applied on the underline dataset, K-NN would be the best choice to predict the customers who will accept the personal loan.

## ---------------------------------------END----------------------------------------------